In [106]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import seaborn as sns

In [294]:
def load_census_data(directory):
    # Load census data from given directory
    df = []
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith('.csv'):
            df_temp = pd.read_csv(os.path.join(directory,filename),skiprows=1)
            df_temp["Year_ID"] = filename[:4] + df_temp['Id2'].map(str)
            df_temp = df_temp.set_index("Year_ID")
            df.append(df_temp)


    census_data = pd.concat(df)
    return census_data

In [295]:
census_data_2013 = load_census_data("data/census_data_2013/")

/Users/admin/miniconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]


In [354]:
estimates = [c for c in census_data_2013.columns if c[:9] == "Estimate;"]
census_estimates_2013 = census_data_2013[estimates]
census_estimates_2013 = census_estimates_2013.replace(to_replace='(X)',value=np.nan)
census_estimates_2013 = census_estimates_2013.fillna(0)
census_estimates_2013 = census_estimates_2013.astype(float)

census_estimates_2013

,Estimate; ANCESTRY - Total population,Estimate; ANCESTRY - Total population - American,Estimate; ANCESTRY - Total population - Arab,Estimate; ANCESTRY - Total population - Czech,Estimate; ANCESTRY - Total population - Danish,Estimate; ANCESTRY - Total population - Dutch,Estimate; ANCESTRY - Total population - English,Estimate; ANCESTRY - Total population - French (except Basque),Estimate; ANCESTRY - Total population - French Canadian,Estimate; ANCESTRY - Total population - German,...,"Estimate; WORLD REGION OF BIRTH OF FOREIGN BORN - Foreign-born population, excluding population born at sea - Latin America","Estimate; WORLD REGION OF BIRTH OF FOREIGN BORN - Foreign-born population, excluding population born at sea - Northern America","Estimate; WORLD REGION OF BIRTH OF FOREIGN BORN - Foreign-born population, excluding population born at sea - Oceania",Estimate; YEAR OF ENTRY - Foreign born,Estimate; YEAR OF ENTRY - Foreign born - Entered 2010 or later,Estimate; YEAR OF ENTRY - Foreign born - Entered before 2010,Estimate; YEAR OF ENTRY - Native,Estimate; YEAR OF ENTRY - Native - Entered 2010 or later,Estimate; YEAR OF ENTRY - Native - Entered before 2010,Estimate; YEAR OF ENTRY - Population born outside the United States
Year_ID,,,,,,,,,,,,,,,,,,,,,
201521001,18852.0,3413.0,17.0,0.0,26.0,144.0,4386.0,362.0,36.0,1612.0,...,94.0,31.0,0.0,243.0,45.0,198.0,61.0,8.0,53.0,304.0
201521003,20355.0,3477.0,10.0,25.0,0.0,424.0,1956.0,336.0,11.0,2599.0,...,123.0,14.0,12.0,238.0,17.0,221.0,289.0,0.0,289.0,527.0
201521005,21761.0,4576.0,0.0,23.0,101.0,158.0,2801.0,368.0,44.0,2818.0,...,125.0,0.0,0.0,198.0,11.0,187.0,118.0,64.0,54.0,316.0
201521007,8256.0,1629.0,10.0,25.0,0.0,117.0,679.0,243.0,1.0,910.0,...,24.0,0.0,0.0,152.0,10.0,142.0,53.0,0.0,53.0,205.0
201521009,42925.0,9496.0,20.0,48.0,61.0,550.0,5639.0,716.0,108.0,3677.0,...,417.0,89.0,0.0,752.0,164.0,588.0,114.0,11.0,103.0,866.0
201521011,11978.0,2967.0,0.0,0.0,0.0,90.0,1122.0,79.0,2.0,1054.0,...,87.0,0.0,0.0,90.0,0.0,90.0,29.0,0.0,29.0,119.0
201521013,27950.0,7465.0,4.0,0.0,5.0,94.0,1347.0,168.0,6.0,1263.0,...,48.0,0.0,0.0,122.0,48.0,74.0,100.0,0.0,100.0,222.0
201521015,124617.0,11675.0,436.0,185.0,218.0,1301.0,12904.0,2309.0,410.0,38474.0,...,1869.0,334.0,73.0,6293.0,997.0,5296.0,988.0,162.0,826.0,7281.0
201521017,20013.0,5919.0,31.0,15.0,41.0,300.0,1949.0,281.0,79.0,2349.0,...,777.0,15.0,5.0,981.0,91.0,890.0,35.0,0.0,35.0,1016.0


In [401]:
def load_drug_data():
    
    census_data = load_census_data("data/census_data_2013/")
    
    drug_data = pd.read_excel('data/drug_data.xlsx', sheet_name = "Data")
    drug_data = drug_data[drug_data["YYYY"] >= 2013]
    drug_data["Year_ID"] = drug_data["YYYY"].map(str) + drug_data["FIPS_Combined"].map(str)
    drug_data = drug_data.set_index("Year_ID")
    fips = drug_data["FIPS_Combined"].map(str).apply(lambda x: x[2:])[~drug_data.index.duplicated()]
    state = drug_data[["State","YYYY"]][~drug_data.index.duplicated()]
    drug_piv = drug_data.pivot(columns="SubstanceName", values=["DrugReports"])

    drug_piv[["TotalDrugReportsCounty","TotalDrugReportsState"]] = drug_data.drop_duplicates(subset=["TotalDrugReportsCounty","TotalDrugReportsState"])[["TotalDrugReportsCounty","TotalDrugReportsState"]]
    
    drug_piv["Total"] = drug_piv["DrugReports"].sum(axis=1)
    drug_piv["FIPS"] = fips
    drug_piv[["State","Year"]] = state
    drug_piv["DrugReportsPerCapita"] = drug_piv["Total"]/census_data["Estimate; RELATIONSHIP - Population in households"]
    return drug_piv

In [403]:
drug_data = load_drug_data()
drug_data.head()

/Users/admin/miniconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]


DrugReports                                    \
SubstanceName 3,4-Methylenedioxy U-47700 3-Fluorofentanyl 3-Methylfentanyl   
Year_ID                                                                      
201321001                            NaN              NaN              NaN   
201321003                            NaN              NaN              NaN   
201321005                            NaN              NaN              NaN   
201321007                            NaN              NaN              NaN   
201321009                            NaN              NaN              NaN   

                                                                 \
SubstanceName 4-Fluoroisobutyryl fentanyl 4-Methylfentanyl ANPP   
Year_ID                                                           
201321001                             NaN              NaN  NaN   
201321003                             NaN              NaN  NaN   
201321005                             NaN              NaN  NaN   
201321007                             NaN              NaN  NaN   
201321009                             NaN              NaN  NaN   

                                                                  \
SubstanceName Acetyl fentanyl Acetylcodeine Acetyldihydrocodeine   
Year_ID                                                            
201321001                 NaN           NaN                  NaN   
201321003                 NaN           NaN                  NaN   
201321005                 NaN           NaN                  NaN   
201321007                 NaN           NaN                  NaN   
201321009                 NaN           NaN                  NaN   

                                     ...                            \
SubstanceName Acryl fentanyl         ...          p-Fluorofentanyl   
Year_ID                              ...                             
201321001                NaN         ...                       NaN   
201321003                NaN         ...                       NaN   
201321005                NaN         ...                       NaN   
201321007                NaN         ...                       NaN   
201321009                NaN         ...                       NaN   

                                                                \
SubstanceName p-methoxybutyryl fentanyl trans-3-Methylfentanyl   
Year_ID                                                          
201321001                           NaN                    NaN   
201321003                           NaN                    NaN   
201321005                           NaN                    NaN   
201321007                           NaN                    NaN   
201321009                           NaN                    NaN   

              TotalDrugReportsCounty TotalDrugReportsState Total FIPS State  \
SubstanceName                                                                 
Year_ID                                                                       
201321001                       73.0               26820.0  17.0  001    KY   
201321003                      108.0               26820.0  23.0  003    KY   
201321005                       50.0               26820.0  25.0  005    KY   
201321007                        NaN                   NaN   2.0  007    KY   
201321009                      306.0               26820.0  63.0  009    KY   

               Year DrugReportsPerCapita  
SubstanceName                             
Year_ID                                   
201321001      2013             0.000963  
201321003      2013             0.001158  
201321005      2013             0.001165  
201321007      2013             0.000245  
201321009      2013             0.001511  

[5 rows x 72 columns]

In [399]:
from sklearn.preprocessing import StandardScaler

standard_census_data = StandardScaler().fit_transform(census_estimates_2013.values)

from sklearn.decomposition import PCA
pca = PCA(n_components=.99, svd_solver='full')

features = pca.fit_transform(standard_census_data)

pca.n_components_

25

In [400]:
target = drug_data["Total"].loc[census_estimates_2013.index].values

/Users/admin/miniconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


In [ ]:
import torch.nn as nn

class NeuralNet(nn.Module):
    def __init__(self, input_size, output_size=1)